**Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
import plotly.express as pe
from sklearn.ensemble import IsolationForest
import ppscore as ps
from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler

# 1. Data Exploration and Preprocessing:

In [2]:
df = pd.read_csv(r"C:\Users\priya\Downloads\EDA2 (\EDA2\adult_with_headers.csv")
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [4]:
df.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

# 2. Encoding Techniques:

In [7]:
#differentiating columns based on their no of unique categories in each 
columns_ohe=[]
columns_le=[]
for i in df.select_dtypes(include=object).columns:
    if len(df[i].unique())<5:
        columns_ohe.append(i)
    else:
        columns_le.append(i)
columns_ohe,columns_le

(['sex', 'income'],
 ['workclass',
  'education',
  'marital_status',
  'occupation',
  'relationship',
  'race',
  'native_country'])

In [8]:
#One-Hot Encoding for categorical values(with less than 5 categories)
le = LabelEncoder()
encoded_df = pd.get_dummies(df,columns=columns_ohe,dtype=int)
for column in columns_le:
    encoded_df[column] = le.fit_transform(df[column])

In [9]:
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [10]:
encoded_df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,sex_ Female,sex_ Male,income_ <=50K,income_ >50K
0,39,7,77516,9,13,4,1,1,4,2174,0,40,39,0,1,1,0
1,50,6,83311,9,13,2,4,0,4,0,0,13,39,0,1,1,0
2,38,4,215646,11,9,0,6,1,4,0,0,40,39,0,1,1,0
3,53,4,234721,1,7,2,6,0,2,0,0,40,39,0,1,1,0
4,28,4,338409,9,13,2,10,5,2,0,0,40,5,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,2,13,5,4,0,0,38,39,1,0,1,0
32557,40,4,154374,11,9,2,7,0,4,0,0,40,39,0,1,0,1
32558,58,4,151910,11,9,6,1,4,4,0,0,40,39,1,0,1,0
32559,22,4,201490,11,9,4,1,3,4,0,0,20,39,0,1,1,0


# Pros and Cons of One-Hot Encoding and Label Encoder
**One-Hot Encoding**
- Pros:

1. No Ordinal Assumption: One-Hot Encoding does not assume any ordinal relationship between categories, making it suitable for nominal data (e.g., colors, types of animals) where no category is more important than another.
2. More Information Retention: By creating binary columns for each category, One-Hot Encoding retains more information about the data. This can improve the model's performance, especially when dealing with non-linear models.
3. Compatibility with Most Models: One-Hot Encoded data is compatible with a wide range of machine learning algorithms, including linear models, tree-based models, and neural networks.

- Cons:

1. High Dimensionality: One-Hot Encoding can significantly increase the dimensionality of the dataset, especially if the categorical variable has many unique categories. This can lead to the "curse of dimensionality," making the model more complex and harder to train.
2. Sparsity: The resulting matrix is often sparse (i.e., containing many zeros), which can lead to increased memory usage and computational costs.
3. Not Suitable for High Cardinality: For categorical variables with a large number of unique categories (high cardinality), One-Hot Encoding may become impractical due to the high number of columns it creates.
Label Encoding

- Pros:

1. Simplicity: Label Encoding is straightforward and easy to implement. It converts each category into a unique integer, which is simple to store and manage.
2. Low Dimensionality: Unlike One-Hot Encoding, Label Encoding does not increase the number of columns in the dataset, which helps keep the feature space compact.
3. Suitable for Ordinal Data: Label Encoding is ideal for ordinal data (e.g., low, medium, high) where the order of categories matters.

- Cons:

1. Imposes Ordinal Relationship: Label Encoding introduces an implicit ordinal relationship between the categories. For nominal data (where there is no natural order), this can lead to misleading results, as the model might interpret one category as more significant than another.
2. Bias in Certain Algorithms: Some machine learning algorithms (e.g., linear models, distance-based models) might be biased by the numerical encoding, leading to suboptimal performance.
3. Not Suitable for Nominal Data: Label Encoding is generally not suitable for nominal data since it can mislead models into assuming an order where there is none.


- **When to Use Which?**
1. One-Hot Encoding:

Best for nominal categorical variables with no inherent order.
Ideal for small to medium-sized datasets where the number of unique categories is manageable.
Preferred when using algorithms that can handle high-dimensional sparse data, like tree-based models (e.g., Random Forest) and neural networks.

2. Label Encoding:

Suitable for ordinal categorical variables where the order matters.
Useful when you need to keep the dataset's dimensionality low, particularly when working with algorithms that can't handle high-dimensional data well.
Can be used when the number of unique categories is small and when using algorithms that don't interpret the numerical labels as ordinal (e.g., decision trees).

# 3. Feature Engineering:

In [11]:
# Create a new feature i.e no of years spent on education
# Mapping education levels to years of education
education_mapping = {
9:12,
10:13,
13:16,
14:18,
11:14,
7:11,
12:14,
6:10,
4:8,
15:21,
5:9,
8:12,
16:22,
3:6,
2:4,
1:0}


# Apply the mapping to create a new feature
encoded_df['years_of_education'] = df['education_num'].map(education_mapping)

In [12]:
# Create age group feature
encoded_df['age_group'] = pd.cut(df['age'], bins=[0, 25, 45, 65, 100], labels=['young', 'middle-aged', 'senior', 'elderly'])

# Display the distribution of the new feature
print(encoded_df['age_group'].value_counts().sum())

32561


In [13]:
encoded_df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,sex_ Female,sex_ Male,income_ <=50K,income_ >50K,years_of_education,age_group
0,39,7,77516,9,13,4,1,1,4,2174,0,40,39,0,1,1,0,16,middle-aged
1,50,6,83311,9,13,2,4,0,4,0,0,13,39,0,1,1,0,16,senior
2,38,4,215646,11,9,0,6,1,4,0,0,40,39,0,1,1,0,12,middle-aged
3,53,4,234721,1,7,2,6,0,2,0,0,40,39,0,1,1,0,11,senior
4,28,4,338409,9,13,2,10,5,2,0,0,40,5,1,0,1,0,16,middle-aged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,2,13,5,4,0,0,38,39,1,0,1,0,14,middle-aged
32557,40,4,154374,11,9,2,7,0,4,0,0,40,39,0,1,0,1,12,middle-aged
32558,58,4,151910,11,9,6,1,4,4,0,0,40,39,1,0,1,0,12,senior
32559,22,4,201490,11,9,4,1,3,4,0,0,20,39,0,1,1,0,12,young


In [14]:
encoded_df['years_of_education']=le.fit_transform(encoded_df['years_of_education'])

In [15]:
encoded_df['age_group']=le.fit_transform(encoded_df['age_group'])

In [16]:
encoded_df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,sex_ Female,sex_ Male,income_ <=50K,income_ >50K,years_of_education,age_group
0,39,7,77516,9,13,4,1,1,4,2174,0,40,39,0,1,1,0,10,1
1,50,6,83311,9,13,2,4,0,4,0,0,13,39,0,1,1,0,10,2
2,38,4,215646,11,9,0,6,1,4,0,0,40,39,0,1,1,0,7,1
3,53,4,234721,1,7,2,6,0,2,0,0,40,39,0,1,1,0,6,2
4,28,4,338409,9,13,2,10,5,2,0,0,40,5,1,0,1,0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,2,13,5,4,0,0,38,39,1,0,1,0,9,1
32557,40,4,154374,11,9,2,7,0,4,0,0,40,39,0,1,0,1,7,1
32558,58,4,151910,11,9,6,1,4,4,0,0,40,39,1,0,1,0,7,2
32559,22,4,201490,11,9,4,1,3,4,0,0,20,39,0,1,1,0,7,3


# 4. Feature Selection

**Isolation Forest To detect Outliers**

In [17]:
iso= IsolationForest()
iso.fit(encoded_df)

IsolationForest()

In [18]:
outliers = iso.predict(encoded_df)

In [19]:
outliers

array([ 1,  1,  1, ...,  1,  1, -1])

In [20]:
encoded_df['outliers'] = outliers
encoded_df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,sex_ Female,sex_ Male,income_ <=50K,income_ >50K,years_of_education,age_group,outliers
0,39,7,77516,9,13,4,1,1,4,2174,0,40,39,0,1,1,0,10,1,1
1,50,6,83311,9,13,2,4,0,4,0,0,13,39,0,1,1,0,10,2,1
2,38,4,215646,11,9,0,6,1,4,0,0,40,39,0,1,1,0,7,1,1
3,53,4,234721,1,7,2,6,0,2,0,0,40,39,0,1,1,0,6,2,1
4,28,4,338409,9,13,2,10,5,2,0,0,40,5,1,0,1,0,10,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,2,13,5,4,0,0,38,39,1,0,1,0,9,1,1
32557,40,4,154374,11,9,2,7,0,4,0,0,40,39,0,1,0,1,7,1,1
32558,58,4,151910,11,9,6,1,4,4,0,0,40,39,1,0,1,0,7,2,1
32559,22,4,201490,11,9,4,1,3,4,0,0,20,39,0,1,1,0,7,3,1


In [21]:
encoded_df['outliers'].value_counts()

 1    27053
-1     5508
Name: outliers, dtype: int64

**PP(Predictive Power) Score**

In [22]:
feature_sel = ps.matrix(encoded_df)
feature_sel

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,age,age,1.0,predict_itself,True,None,0.0000,1.000000,None
1,age,workclass,0.0,regression,True,mean absolute error,0.7426,0.875681,DecisionTreeRegressor()
2,age,fnlwgt,0.0,regression,True,mean absolute error,75872.1862,77535.141544,DecisionTreeRegressor()
3,age,education,0.0,regression,True,mean absolute error,2.7590,2.806164,DecisionTreeRegressor()
4,age,education_num,0.0,regression,True,mean absolute error,1.8530,1.898306,DecisionTreeRegressor()
...,...,...,...,...,...,...,...,...,...
395,outliers,income_ <=50K,0.0,regression,True,mean absolute error,0.2424,0.353451,DecisionTreeRegressor()
396,outliers,income_ >50K,0.0,regression,True,mean absolute error,0.2424,0.353451,DecisionTreeRegressor()
397,outliers,years_of_education,0.0,regression,True,mean absolute error,1.4470,1.456172,DecisionTreeRegressor()
398,outliers,age_group,0.0,regression,True,mean absolute error,0.6874,0.743005,DecisionTreeRegressor()


# Comparing PPS and Cororelation Matrix
- The PPS is a score between 0 and 1 that indicates how well one feature can predict another. Unlike correlation, which only measures linear relationships, the PPS can detect and quantify non-linear relationships.
- PPS is asymmetric, meaning that PPS(X -> Y) may not be equal to PPS(Y -> X).
- The correlation matrix is a table showing correlation coefficients between pairs of variables. Correlation coefficients range from -1 to 1, with 1 meaning a perfect positive linear relationship, -1 meaning a perfect negative linear relationship, and 0 meaning no linear relationship.